### Abstract

We implement the SGHMC described in paper Stochastic Gradient Hamiltonian Monte Carlo by Chen et al (2014). This specific HMC largely decreases the computation complexity by applying a stochastic gradient calcuted on minibatchs of data with noise and counterbalances that noise by a friction term. (插入). We test the behavior of SGHMC on two examples compared to other two MCMC methods.One of them is the normal HMC from pyhmc package, and the another one is from pystan package.There is an up-to-date Github repository at (网址). The instruction of installation and explaination of function will be avaliable in README file. 

### Introduction

Hamiltonian Monte Carlo (HMC) (Duane et al., 1987; Neal, 2010) is an advanced Markov Chain Monte Carlo (MCMC) sampling method. Rather than using a traditionally statistical way to find a proposal distribution, HMC introduce the definition of potential energy and kinetic energy from physics to find a proposal distribution. However, there is a limitation for HMC, which is the requirement to compute the gradient of the potential energy function during a sampling process. This problem could be addressed when the data size is small. Nonetheless, with the increasing data size in real world analysis, calculating the gradient of potential energy for whole dataset is very costly and time consuming. In order to address the weakness in HMC, the paper Stochastic Gradient Hamiltonian Monte Carlo by Chen et al (2014) develops an advanced version of HMC to sidestep the inner limitation.Stochastic Gradient Hamiltonian Monte Carlo (SGHMC) works the same way as HMC but is different from their inner computation. Instead of calculating the gradient of potential energy function for whole dataset, SGHMC applies a stochastic gradient on minibatches of data with a certain degree noise and then counteracts that noise with a “friction” term.

SGHMC can be appliled in any situation where MCMC methods are needed and it also takes advantage of producing low correlation samples during the sampling procedure, while also decreasing the computation cost especially when data size is large. This allows a more efficient exploration in the state space. One limitation of SGHMC is that users need to approximate many variables initially compared to other algorithms and different setups may lead to different target distributions. Another limitation is that the stochastic gradient calculated based on minibatches of data may be inappropriate since the observation size is not enough.

### Algorithm Description

The derivation of SGHMC is very complex. Let us start from the basic HMC formula. With the help of auxiliary variables in physics system, the Hamiltonian function is defined by:

$$
H(\theta,r)=U(\theta)+\frac{1}{2}r^{T}M^{-1}r^{-T}
$$

Where $U(\theta)$ is the potential energy defined by $U(\theta)=-\sum_{x \in D}logp(x|\theta)-logp(\theta)$ given a set of independent obervations $x \in D$ while r and mass matrix M together define the kinectic energy term. Then, the unit change in $\theta$ and $r$ is:

\begin{align}
d\theta &= M^{-1}rdt \\
dr &= -\nabla U(\theta)dt
\end{align}

SGHMC introduces a stochastic gradient based on minibatch of data $\tilde D$ with noise into system and counterbalance that noise by a friction term. Then the stochastic gradient is defined by $\nabla \tilde{U}(\theta)=-\frac{|D|}{|\tilde{D}|}\sum_{x \in \tilde{D}}\nabla logp(x|\theta)-\nabla logp(\theta)$ Thus the unit change in $\theta$ and $r$ is defined by:

\begin{align}
d\theta&=M^{-1}rdt \\
dr&=-\nabla U(\theta)dt-BM^{-1}rdt+N(0,2Bdt)
\end{align}

For simplicity, $B(\theta)$ is abbreviated to $B$. $N(0,2Bdt)$ is the noise approximated by a normal distribution where $B(\theta)=\frac{1}{2}\epsilon V(\theta)$ is the defusion matrix contributed by gradient noise. $\epsilon$ is the step size, and it takes similar function with learning rate in gradient descent. Thus, it is a very small number. $V(\theta)$ is the covariane of stochastic gradient noise. $-BM^{-1}rdt$ is the friction term. However, in practice we rarely know the exact noise moel $B$. We use $\hat{B}$ instead to approxiamte noise model. As a result, a new friction term $C\succeq \hat{B}$ is introduced. The new equation of dr is:

$$
dr=-\nabla U(\theta)dt-CM^{-1}rdt+N(0,2(C-\hat{B})dt)+N(0,2Bdt)
$$

Finally,the algorithm is:

initialize $(\theta_0, r_0,\epsilon, M,\hat{B},B,C)$

when t=1,2,3......

\begin{align}
\theta_i&=\theta_{i-1}+\epsilon_tM^{-1}r_{i-1}\\
r_i&=r_{i-1}-\epsilon_t \nabla \tilde {U}(\theta_i)-\epsilon_t CM^{-1}r_{i-1}+N(0,2(C-\hat{B})\epsilon_t)
\end{align}

### Comparative analysis

In this part, we will conduct comparative analysis on differnet sampling methods. We decide to use Pyhmc package, Pystan package(a no U turn implementation of HMC), and the SGHMC package written by ourselfs as our three MCMC methods. The first example is the one that has already been disscussed in the paper but we want to reimplement it to get a same answer. In the first example, $U(\theta)$ is defined by $U(\theta)=-2\theta^2+\theta^4$. The second one is a mix normal model and $U(\theta)$ is defined by $U(\theta)=0.5N(-3,1)+0.5N(3,1)$. In both examples, we need to set up some initail values for SGHMC.
 
In the first example, we set $\hat{V}$, an approxiamtion of $V(\theta)$, to the product of a one dimensional identity matrix and 0, which is a one dimensional 0. $\epsilon$ is equal to 0.1, batch_size is 1, and friction term $C$ is $I(1)*2*\epsilon$, which $I(1)$ is a one dimensional identity matrix. The way we set up the initial value above is in order to conform with the original values in paper. we choose the effective sample size to be 6000. The following graphs are density plots of sampled $\theta$ in three different methods. As we can see, the denity plots of Pystan and SGHMC are very similar to the true distribution. However, the graph of pyhmc shows that it does not sample a distribution that is close to true distribution compared to other two. Then we try to increase the sample size in pyhmc to 10000 or even bigger. The pyhmc graph is still similar to original one, which indicates pyhmc is not as good as the other two. 
![pyhmc](./example1_pyhmc.png)
![pystan](./example1_pystan.png)

For the initial setup in example2, we set both friction term $C$ and $\hat{V}$ to two dimensional identity matrix. The $\epsilon$ is equal to 0.01 and the initial value for $\mu$ is $(0,0)$. The effective sample size is also 6000. The behavior of pyhmc is quiet poor. It takes relatively long time to run the sampling process and the outcome shows that pyhmc doesn't explore the state space at all. The sampled $\mu$ values are far away from the true $\mu$. If we set $\epsilon$ to value greater than 0.03, there are even no plots shown in kernel density plot. For pystan and SGHMC, even though their graphs look a little bit different, they actually represent the same thing since two circles in pystan graph are symmetric. That means $\theta$ sometimes jumps from -3 to 3 or 3 to -3 in the sampling process, which is reasonable in sampling procedure. As a result, they both successfully sample the true distrbution. 
![pystan](./example2_pystan.png)
![pyhmc](./example2_pyhmc.png)

Pyhmc behaves poor when it is running on both examples. Compared to pyhmc, pystan and SGHMC wrritten by ourselves behave better. Despite the comparatible results shown above, pystan is acutually better than SGHMC. Pystan is quite simple to use since we only need to pass some distributions and data into function. However, SGHMC requires users to approximate many variables and unprecise approximations will lead to serious erros. In general, our own developed SGHMC package is good at first glance.